# Model Context Protocol (MCP) Concepts

The **Model Context Protocol (MCP)** is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Think of it as a universal adapter that allows AI assistants to connect to various data sources and tools.

> 📝 **Hands-on Exercises**: After reviewing these concepts, complete the exercises in [EXERCISES.md](EXERCISES.md).

## Key Benefits
- **Standardization**: One protocol to connect to many tools
- **Security**: Controlled access to resources
- **Flexibility**: Works with any LLM provider

## MCP Architecture

```
+-----------------+     +-----------------+     +-----------------+
|   AI Agent      |---->|   MCP Client    |---->|   MCP Server    |
|   (Host)        |     |                 |     |   (Tools)       |
+-----------------+     +-----------------+     +-----------------+
```

### Components
1. **Host**: The AI application (e.g., Claude, ChatGPT integration, or custom agent)
2. **Client**: Connects to MCP servers on behalf of the host
3. **Server**: Exposes tools, resources, and prompts

## Transport Types

MCP supports two primary transport mechanisms:

### 1. STDIO (Standard Input/Output)
- Used for **local** MCP servers
- Server runs as a subprocess
- Communication via stdin/stdout
- Best for: Local tools, CLI applications

### 2. HTTP/SSE (Server-Sent Events)
- Used for **remote** MCP servers
- Server runs as a web service
- Communication via HTTP requests and SSE
- Best for: Cloud services, shared tools

## Workshop Projects Overview

| Project | Type | Transport | Description |
|---------|------|-----------|-------------|
| McpLocal | Local | STDIO | .NET MCP server with Config and Ticket tools |
| RemoteServer | REST API | HTTP | Backend REST API for tickets |
| McpBridge | Remote | HTTP/SSE | MCP server that calls REST API |
| McpAgentClient | Client | Both | AI agent that consumes MCP servers |

## Setup: Install Required NuGet Packages

Run the following cell to install the required packages for MCP and Azure OpenAI integration.

In [ ]:
#r "nuget: ModelContextProtocol, 0.3.0-preview.1"
#r "nuget: Microsoft.Extensions.Hosting, 10.0.0-preview.4.25258.110"
#r "nuget: Microsoft.Extensions.Configuration.Json, 10.0.0-preview.4.25258.110"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 10.0.0-preview.4.25258.110"
#r "nuget: Azure.AI.OpenAI, 2.3.0-beta.1"
#r "nuget: Azure.Identity, 1.13.2"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.7.0-preview.1.25356.2"
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.25171.1"

using System;
using System.IO;
using System.Text.Json;
using System.ClientModel;
using Microsoft.Extensions.Configuration;
using ModelContextProtocol.Client;
using Azure.AI.OpenAI;
using Azure.Identity;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;

Console.WriteLine("✅ Packages loaded successfully!");

## Configuration Setup

Configure your Azure OpenAI credentials. You can either:
1. Set them directly in the cell below, or
2. Use the `appsettings.Local.json` file in the `dotnet` folder

In [ ]:
// Option 1: Set credentials directly (uncomment and fill in)
// var endpoint = "https://your-resource.openai.azure.com/";
// var apiKey = "your-api-key";
// var deploymentName = "gpt-4o-mini";

// Option 2: Load from appsettings.Local.json
var configPath = Path.GetFullPath(Path.Combine(Directory.GetCurrentDirectory(), ".."));
var configuration = new ConfigurationBuilder()
    .SetBasePath(configPath)
    .AddJsonFile("appsettings.Local.json", optional: true)
    .AddEnvironmentVariables()
    .Build();

var endpoint = configuration["AZURE_OPENAI_ENDPOINT"] 
    ?? configuration["AzureOpenAI:Endpoint"] 
    ?? throw new InvalidOperationException("Azure OpenAI endpoint not configured");
    
var apiKey = configuration["AZURE_OPENAI_API_KEY"] 
    ?? configuration["AzureOpenAI:ApiKey"];
    
var deploymentName = configuration["AZURE_OPENAI_DEPLOYMENT_NAME"] 
    ?? configuration["AzureOpenAI:DeploymentName"] 
    ?? "gpt-4o-mini";

Console.WriteLine($"✅ Configuration loaded");
Console.WriteLine($"   Endpoint: {endpoint}");
Console.WriteLine($"   Deployment: {deploymentName}");
Console.WriteLine($"   Auth: {(string.IsNullOrEmpty(apiKey) ? "Azure CLI" : "API Key")}");

## Create Azure OpenAI Client

Create the Azure OpenAI client with the configured credentials.

In [ ]:
AzureOpenAIClient azureOpenAIClient;

if (!string.IsNullOrWhiteSpace(apiKey))
{
    azureOpenAIClient = new AzureOpenAIClient(new Uri(endpoint), new ApiKeyCredential(apiKey));
    Console.WriteLine("✅ Created Azure OpenAI client with API Key authentication");
}
else
{
    azureOpenAIClient = new AzureOpenAIClient(new Uri(endpoint), new AzureCliCredential());
    Console.WriteLine("✅ Created Azure OpenAI client with Azure CLI authentication");
}

## Demo 1: Connect to Local MCP Server (STDIO)

Connect to the local .NET MCP server using STDIO transport. This server provides configuration and ticket management tools.

In [ ]:
// Get the path to the McpLocal project
var solutionDir = Path.GetFullPath(Path.Combine(Directory.GetCurrentDirectory(), ".."));
var mcpLocalProject = Path.Combine(solutionDir, "McpLocal", "McpLocal.csproj");

Console.WriteLine($"📁 Solution directory: {solutionDir}");
Console.WriteLine($"📁 MCP Local project: {mcpLocalProject}");
Console.WriteLine();

// Create MCP client for local .NET server using STDIO transport
var localTransport = new StdioClientTransport(new StdioClientTransportOptions
{
    Name = "LocalDotNetMcpServer",
    Command = "dotnet",
    Arguments = ["run", "--project", mcpLocalProject]
});

Console.WriteLine("🔄 Connecting to Local MCP Server...");
var localMcpClient = await McpClientFactory.CreateAsync(localTransport);
Console.WriteLine("✅ Connected to Local MCP Server!");

// List available tools
var localTools = await localMcpClient.ListToolsAsync();
Console.WriteLine($"\n🔧 Available tools ({localTools.Count}):");
foreach (var tool in localTools)
{
    Console.WriteLine($"   - {tool.Name}: {tool.Description}");
}

## Create AI Agent with Local MCP Tools

Create an AI agent that can use the local MCP server tools.

In [ ]:
var localAgent = azureOpenAIClient
    .GetChatClient(deploymentName)
    .AsIChatClient()
    .CreateAIAgent(
        instructions: "You are a configuration management assistant. Help users get and update configurations using the available MCP tools.",
        tools: [.. localTools.Cast<AITool>()]);

var localThread = localAgent.GetNewThread();
Console.WriteLine("✅ AI Agent created with Local MCP tools!");

## Test: Query Configurations

Ask the AI agent to get all configurations from the local MCP server.

In [ ]:
var prompt = "Get all configurations";
Console.WriteLine($"🗣️ You: {prompt}");
Console.WriteLine();

var response = await localAgent.RunAsync(prompt, localThread);
Console.WriteLine($"🤖 Agent: {response}");

## Test: Update Configuration

Ask the AI agent to update a configuration value.

In [ ]:
var updatePrompt = "Update feature.darkMode to true";
Console.WriteLine($"🗣️ You: {updatePrompt}");
Console.WriteLine();

var updateResponse = await localAgent.RunAsync(updatePrompt, localThread);
Console.WriteLine($"🤖 Agent: {updateResponse}");

## Demo 2: Connect to Remote MCP Server (HTTP/SSE)

Connect to the remote MCP Bridge server using HTTP/SSE transport.

**Prerequisites:** Before running this cell, start the following servers in separate terminals:
```bash
# Terminal 1: Start REST API
cd RemoteServer
dotnet run  # Starts on port 5060

# Terminal 2: Start MCP Bridge
cd McpBridge
dotnet run  # Starts on port 5070
```

In [ ]:
// Create MCP client for remote server using SSE transport
var remoteTransport = new SseClientTransport(new SseClientTransportOptions
{
    Name = "McpBridge",
    Endpoint = new Uri("http://localhost:5070/sse")
});

Console.WriteLine("🔄 Connecting to MCP Bridge at http://localhost:5070/sse...");

try
{
    var remoteMcpClient = await McpClientFactory.CreateAsync(remoteTransport);
    Console.WriteLine("✅ Connected to MCP Bridge!");

    // List available tools
    var remoteTools = await remoteMcpClient.ListToolsAsync();
    Console.WriteLine($"\n🔧 Available tools ({remoteTools.Count}):");
    foreach (var tool in remoteTools)
    {
        Console.WriteLine($"   - {tool.Name}: {tool.Description}");
    }
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Failed to connect: {ex.Message}");
    Console.WriteLine("\n💡 Make sure RemoteServer and McpBridge are running:");
    Console.WriteLine("   Terminal 1: cd RemoteServer && dotnet run");
    Console.WriteLine("   Terminal 2: cd McpBridge && dotnet run");
}

## Cleanup

Dispose of the MCP client connections when done.

In [ ]:
if (localMcpClient != null)
{
    await localMcpClient.DisposeAsync();
    Console.WriteLine("✅ Local MCP client disposed");
}

## Local vs Remote MCP: When to Use Which?

### Use Local (STDIO) When:
- Tool needs access to local files
- Low latency is critical
- No network dependency required
- Single-user scenarios

### Use Remote (HTTP/SSE) When:
- Tool needs to access remote APIs
- Multiple users need the same tools
- Centralized logging/monitoring needed
- Tool requires server-side resources

## Authentication Options

The McpAgentClient supports multiple authentication methods for Azure OpenAI:

| Method | Configuration | Use Case |
|--------|---------------|----------|
| **API Key** | `AzureOpenAI:ApiKey` | Simple setup, development |
| **Service Principal** | `TenantId`, `ClientId`, `ClientSecret` | Production, CI/CD |
| **Managed Identity** | `UseManagedIdentity: true` | Azure-hosted applications |
| **Azure CLI** | Default (no config) | Local development |

Configuration priority: Environment variables > appsettings.Local.json

## Summary

| Concept | Description |
|---------|-------------|
| **MCP** | Protocol for AI-to-tool communication |
| **STDIO Transport** | Local subprocess communication |
| **HTTP/SSE Transport** | Remote web service communication |
| **Tools** | Functions exposed by MCP servers |
| **McpClientFactory** | Creates MCP client connections |
| **[McpServerTool]** | Attribute to mark a method as an MCP tool |
| **[McpServerToolType]** | Attribute to mark a class containing MCP tools |